#### Getting Started with LangChain: Creating a Simple Prompt and Model Chain

LangChain makes building applications powered by language models seamless and efficient. One of its core features is the ability to create prompt-and-model chains, enabling you to link structured input prompts with specific models to generate meaningful outputs. In this post, we’ll walk through creating a simple LangChain prompt and model chain, perfect for beginners looking to understand the basics of this powerful framework. Let’s dive in!

I picked up Llama models because, from a pay-for-API perspective, they eliminate the recurring costs associated with proprietary APIs like OpenAI and Anthropic. I picked up Tiny Llama because of compute limitations. Nevertheless, this should not stop me from using it to build intuition for learning, experimenting, and understanding the fundamentals of working with language models. By integrating it with LangChain's RAG (Retrieval-Augmented Generation) agents, I can test and explore advanced workflows, like combining knowledge retrieval with lightweight models, even in resource-constrained environments.


#### Create Custom LLM wrapper

LangChain provides integraton to llama by [Llamafile](https://python.langchain.com/docs/integrations/llms/llamafile/). Which i would take up in separate post. I have created a custom wrapper for tinyllama for this example. The instructions to writing custom implementation are [here](https://python.langchain.com/v0.1/docs/modules/model_io/llms/custom_llm/)

The __call method takes in prompt and makes an API call to the ollama endpoint (hosted on docker). 
And the small for loop is to iterate over the response line by lline and stop when tiny llama is done generating.


In [2]:
import json
import requests
from typing import Optional
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM

In [3]:
class OllamaLLM(LLM):
    """
    A custom LLM integration for Ollama's API.
    """

    model: str = "tinyllama"  # Default model
    base_url: str = "http://localhost:11434"  # Default Ollama endpoint

    def _call(self, prompt: str, stop: Optional[list] = None) -> str:
        """
        Handles the interaction with the Ollama API.
        """
        payload = {"model": self.model, "prompt": prompt}
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json=payload,
                stream=True,  # Enable streaming
            )
            response.raise_for_status()

            # Process streamed responses
            full_response = ""
            for line in response.iter_lines(decode_unicode=True):
                if line:  # Skip empty lines
                    try:
                        data = json.loads(line)
                        #print("Streaming JSON Object:", data)  # Debugging
                        full_response += data.get("response", "")
                        if data.get("done", False):  # Stop when done
                            break
                    except json.JSONDecodeError as e:
                        print(f"Failed to decode line: {line}. Error: {e}")
                        continue

            return full_response
        except requests.RequestException as e:
            raise ValueError(f"Error communicating with Ollama API: {e}")
        except ValueError as e:
            raise ValueError(f"Error processing the response: {e}")

    @property
    def _identifying_params(self) -> dict:
        """Returns identifying parameters for serialization."""
        return {"model": self.model, "base_url": self.base_url}

    @property
    def _llm_type(self) -> str:
        """Type of the LLM."""
        return "ollama"

# Instantiate the Ollama LLM
ollama_llm = OllamaLLM(model="tinyllama", base_url="http://localhost:11434")

# Create a PromptTemplate
prompt = PromptTemplate(
    input_variables=["topic"],  # Variables to inject
    template="Explain {topic} in simple terms.",
)

# chain prompt and llm
chain = prompt | ollama_llm
# Run the chain with a specific input
result = chain.invoke({"topic": "physics"})
print("LangChain Response:", result)


LangChain Response: Physics is the study of how matter and energy behave at all scales, from subatomic particles to galaxies, and encompasses many different branches of science. Here are some key concepts and terminology that you might encounter:

1. Matter (or substances): The physical components of objects such as atoms, molecules, or gases.

2. Energy: A fundamental aspect of matter, energy is the ability to do work or move something. It can come in different forms such as kinetic, potential, and thermal.

3. Force: This is a force that acts over short distances between two objects (or two objects acting on one another). Force is measured by mass times acceleration. For example, gravity causes objects to pull on each other.

4. Motion: The state of being in motion, whether in space or time.

5. Pressure: An amount of force applied per unit area. Pressure can occur when two objects are held apart, or when fluids flow through spaces.

6. Force-momentum diagram: A graph showing the rel